# Section 1: Model Analysis & Interpretation #

## 1.1 Introduction & Motivation ##
Since American options can be exercised anytime before expiration, it is difficult to determine the value of these options based off a simple formula. In this project, I will be exploring how different machine learning models can be used to price American options.